## Scientific Computing 2018: Homework Assignment 1 
Due Thursday October 11, 2018, 12:00

### Problem 1 (2 points)
Under assumptions of Amdahl's law, suppose that 60% of a program are perfectly parallelizable, and the rest is not parallelizable. 
1. What is the maximum speedup achievable by parallelization? 

$$\lim_{p\to\infty}{S_p} = \lim_{p\to\infty}\frac{1}{\alpha + \frac{1 - \alpha}{p}} = 2.5  $$

2. Suppose that we have obtained speedup 2 (by using a suitable number of processes). What is the efficiency of this parallelization? 

$$ E = \frac{S}{N} $$

$$ S = \frac{T_1}{T_N} = \frac{1}{\alpha + \frac{1 - \alpha}{N}} = 2 $$

$$ N = 6$$

$$ E = \frac{1}{3} $$


### Problem 2 (2 points)
Write a Python or C/C++ program that uses **MPI reduce** to find the largest file in terms of the  number of lines among all .txt files in the working directory. The program must be callable in the form `mpirun -np <N> python linecount.py` (in the case of the Python version) or `mpirun -np <N> linecount.exe` (the C/C++ version), where `<N>` is the user-defined number of processes. The program is expected to first somehow distribute the files found in the current directory to the processes, then each process should count the lines in the files assigned to it, and finally the result should be MPI-reduced and printed out. 

In [9]:
from mpi4py import MPI
import os
import fnmatch

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

path = "/Users/anna/Documents/Papka/"
files = os.listdir(path)


if rank == 0:
    for each_file in files:
        file_array = [files[i: len(files): size] for i in range(size)]
else:
    file_array = None

data = comm.scatter(file_array, root=0)
#print ('rank', rank, 'has data:', data)


size_file = -1
name = ''

for each_file in data:
    if fnmatch.fnmatch(each_file, '*.txt'):
        with open(path + each_file) as file_name:
            num_line = sum(1 for _ in file_name)
        if num_line > size_file:
            size_file = num_line
            name = each_file


big = comm.reduce([size_file, name], op = MPI.MAXLOC, root = 0)

if rank == 0:
    print ('Max size : ', big)

Max size :  [399, 'data_group4.txt']


### Problem 3 (2 points)
Solve the Distinct Substrings problem at Sphere online judge: http://www.spoj.com/problems/DISUBSTR/. Provide code passing the test of the judge. Explain how your code works and theoretically estimate the complexity of the algorithm (as $O(f(N))$, where $f(N)$ is some function of the length of the input string). 

In [12]:
import numpy as np
import sys

def invPerm(p):
    '''Invert the permutation p'''
    s = np.empty(p.size, p.dtype)
    s[p] = np.arange(p.size)
    return s


def getSA(A):
    if not type(A) is np.ndarray:
        A = np.array(list(A))
    N = len(A) 
    M = int(np.ceil(np.log2(N)))+1   # number of iterations
    
    # auxiliary arrays; row m stores results after m'th step:
    
    # positions of sorted length-(2**m) sequences in A
    P = np.zeros((M,N), dtype=int) 
    
    # rank (0, 1, etc.) of sorted length-(2**m) sequences after sorting
    Q = np.zeros((M,N), dtype=int)     
    
    # rank of sorted length-(2**m) sequences at its starting position in A;
    # padded by 0 on the right
    R = np.zeros((M,3*N), dtype=int) 

    for k in range(M):
        if k == 0:
            P[0] = np.argsort(A)
            Q[0][1:] = np.cumsum(A[P[0]][1:] != A[P[0]][:-1])
            R[0][:N] = Q[0][invPerm(P[0])]
        else:
            offset = 2**(k-1)
            r = np.lexsort((R[k-1, P[k-1]+offset], R[k-1, P[k-1]]))
            P[k] = P[k-1][r]
            # k'th rank increases iff (k-1)'th rank increases at least for one element of the pair    
            Q[k][1:] = np.cumsum(np.logical_or(R[k-1][P[k]][1:] != R[k-1][P[k]][:-1], 
                                          R[k-1][P[k]+offset][1:] != R[k-1][P[k]+offset][:-1]))
            R[k][:N] = Q[k][invPerm(P[k])]
            
            # early stopping if suffixes already fully sorted (max rank is N-1)
            if Q[k][-1] == N-1: 
                break
    
    SA = P[k]
    return SA, P[:k+1], Q[:k+1], R[:k+1]  

'''
def input_string(T):
    string_array = []
    for i in range(T):
        new_string = input()
        string_array.append(new_string)
    return string_array
'''

def getLCP(SA, R):
    (M, N) = R.shape
    LCP = np.zeros((len(SA)-1,),dtype=int)
    for m in range(M-1)[::-1]:
        t = (R[m][SA[1:]+LCP] == R[m][SA[:-1]+LCP]).astype(int)
        LCP += (2**m)*t
    return LCP

def main():
    T = int(input())
    #new_strings = input_string(T)
    strings = []
    for i in range(T):
        A = input() + '$'
        strings.append(A)
    for j in range(T):
        B = strings[j]
        SA, _, _, R = getSA(B)
        LCP = getLCP(SA, R)
        N = len(B)
        print((N - 1 - SA).sum()- LCP.sum())
   

if __name__ == '__main__':
    main()  

1
CCCCC
5


The code is based on the functions _getSA_ and _getLCP_, which are described(their work) in the file SuffixArrays_2018.ipynb.

For finding the answer we should calculate 
$\sum_{i=0}^{N} (N - 1 -SA[i])$ - $\sum_{i=0}^{n-1} (LCP[i])$, where N is len of our string. 

The complexity of the algorithm is $O(Nlog^2N)$.

### Problem 4 (2 points)
Suppose that we want to distribute $N$ personal projects to $N$ students. Assume that each student $(k)_{k=0}^{N-1}$ has a list of his/her preferences for the projects, expressed as a vector $\mathbf r_k$ of integer ranks assigned to each project. Ranks vary between 0 and $N-1$ without repetitions, the **lower** the rank the **more preferable** the project. (For example, the first student's ranks are $\mathbf r_0 = [2,1,0]$, the second's $\mathbf r_1 = [0,2,1]$ and the third $\mathbf r_2 = [0,1,2]$). We want to distribute the projects so as to maximize the total preference, i.e., if $n_k$ denotes the project assigned to the $k$'th student, we want to make $f = \sum_{k=0}^{N-1} \mathbf r_k[n_k]$ as small as possible. (In the example above the optimal distribution is $n_0=2, n_1=0, n_2=1$, which gives $f=1$).  
  * Come up with an algorithm optimizing the distribution and implement it in a Python or C/C++ program. The algorithm should accept the preference vectors and output a recommended distribution $(n_k)_{k=1}^N$. The algorithm need not find the best solution, but is expected to generally produce better solutions than would have been obtained by randomly distributing the projects. The algorithm should be reasonably fast, say run in not more than a few seconds for $N=30$. 
  * Compare experimentally your algorithm with the trivial algorithm producing a random distribution. To this end, perform $M=1000$ experiments in each of which 1) random preference vectors for $N=30$ students and projects are generated; 2) the objective function $f$ is evaluated for both algorithms. After finishing all the experiments, plot the two respective distributions of the obtained $M$ values of $f$ and compute the mean values of $f$ for both algorithms. 

In [ ]:
from munkres import Munkres, print_matrix
import numpy as np
import random

def munkrs(matrix):
    m = Munkres()
    indexes = m.compute(matrix)
    total = 0
    for row, column in indexes:
        value = matrix[row][column]
        total += value
    return total

def get_matrix(N):
    m = []
    for i in range(N):
        m.append(random.sample(range(N), N))
    return m

def rand(N, m):
    x = random.sample(range(N), N)        
    y = random.sample(range(N), N)
    sums = 0
    for x_i, y_i in zip(x, y):
        sums += m[x_i][y_i]
    return sums

N = 5
m = [[], []]

matr = get_metrix(N)
ran = rand(N, m)
mun = munkrs(matr)

  
### Problem 5 (2 points)
Suppose that we have developed an algorithm that is supposed to generate independent (quasi-)random numbers uniformly distributed in the interval $[0,1]$. To test our algorithm, we perform a series of experiments. In each experiment, we generate $N=10^3$ numbers $(x_n)_{n=1}^N$ with our algorithm, and compute the minimum distance $r=\min_{1 \le n < m\le N}|x_n-x_m|$ between them. We observe that in more than 90% of such experiments we obtain $r<10^{-5}$. Does this observation contradict the hypothesis of generating independent uniformly distributed random numbers? Explain your answer.